# Fetching Weather Data and Uploading to AWS S3

This guide provides steps for fetching weather data using the OpenWeatherMap API and then uploading the data to an Amazon S3 bucket using Python.

## Step 1: Obtain an API Key from OpenWeatherMap

1. Register on the [OpenWeatherMap website](https://openweathermap.org/) and create an account.
2. Find and copy the API key from your account dashboard.

## Step 2: Write Python Function to Fetch Weather Data

In [1]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime

def get_weather_data(city, api_key):
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {'q': city, 'appid': api_key}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch weather data")

## Step 3: Set Up AWS Credentials for S3 Access
* Install Boto3 using pip install boto3.
* Configure AWS credentials (AWS Access Key ID and Secret Access Key).

In [2]:
!pip install boto3

## Step 4: Write Python Function to Upload Data to S3


In [3]:
import boto3
import json
import aws_credentials

def upload_to_s3(bucket_name, file_name, data, data_type):
    s3 = boto3.client('s3', 
                  aws_access_key_id=aws_credentials.AWS_ACCESS_KEY_ID, 
                  aws_secret_access_key=aws_credentials.AWS_SECRET_ACCESS_KEY)
    if data_type == 'JSON':
        s3.put_object(Bucket=bucket_name, Key=file_name, Body=json.dumps(data))
    elif data_type == 'CSV':
        s3.put_object(Bucket=bucket_name, Key=file_name, Body=data)

## Step 5: Combine the Functions in a Script


In [4]:
# Main execution
api_key = "2f7534325f1c015536373e3892bd720f"   # Replace with your API key
city = "London"  # Replace with desired city
bucket_name = "ia-lab-03"  # Replace with your S3 bucket name
file_name = "weather_data.json"

try:
    weather_data = get_weather_data(city, api_key)
    upload_to_s3(bucket_name, file_name, weather_data, "JSON")
    print("Data uploaded successfully to S3")
except Exception as e:
    print(f"An error occurred: {e}")

Data uploaded successfully to S3


# Assignment: Groups to Convert JSON to CSV

Step 1. Take the JSON output and convert it to a Dataframe using pandas
Step 2. Now upload the CSV file to the 'lab-03' S3 bucket in the cloud with the following naming convention: <your group name>_weather_date_london_<datetimestamp>.csv


In [5]:
def flatten_json(json_obj, prefix=''):
    """
    Flatten a JSON object into a dictionary with flattened keys.

    Args:
        json_obj (dict): The JSON object to flatten.
        prefix (str): Prefix to prepend to flattened keys.

    Returns:
        dict: Flattened dictionary.
    """
    flat_dict = {}  # Initialize an empty dictionary to store flattened key-value pairs
    for key, value in json_obj.items():  
        if isinstance(value, dict):  
            flat_dict.update(flatten_json(value, prefix + key + '_'))  
        elif isinstance(value, list):  
            for i, item in enumerate(value):  
                flat_dict.update(flatten_json(item, prefix + key + '_'))  
        else:  
            flat_dict[prefix + key] = value  # Add the key-value pair to the flattened dictionary

    return flat_dict  # Return the flattened dictionary

# Flatten the JSON-like data
dicti = flatten_json(weather_data)

# Create DataFrame from flattened dictionary
weather_df = pd.DataFrame([dicti])

# Print DataFrame
weather_df

,coord_lon,coord_lat,weather_id,weather_main,weather_description,weather_icon,base,main_temp,main_feels_like,main_temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,-0.1257,51.5085,804,Clouds,overcast clouds,04n,stations,287.24,286.92,286.27,...,1712360257,2,2091269,GB,1712381017,1712428896,3600,2643743,London,200


In [6]:
# Convert DataFrame to CSV format in memory
csv_buffer = StringIO()
weather_df.to_csv(csv_buffer, index=False)
csv_data = csv_buffer.getvalue()

# Generate timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Modify file name to include timestamp
file_name = f"group_06_weather_date_london_{timestamp}.csv"

try:
    upload_to_s3(bucket_name, file_name, csv_data, 'CSV' )
    print("Data uploaded successfully to S3")
except Exception as e:
    print(f"An error occurred: {e}")

Data uploaded successfully to S3
